In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('../data/processed/casas.csv')
df.head()

,tamanho,ano,garagem,preco
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [33]:
X = df.drop('preco', axis = 1)
y = df['preco'].copy()

In [34]:
X.head()

,tamanho,ano,garagem
0,159.0,2003,2
1,117.0,1976,2
2,166.0,2001,2
3,160.0,1915,3
4,204.0,2000,3


In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [37]:
X_train.shape

(1022, 3)

In [38]:
X_test.shape

(438, 3)

## Mlflow

In [51]:
import mlflow

In [73]:
mlflow.set_experiment('house-prices-eda')
# EDA = Análise Exploratória de Dados
# Estamos iniciando um experimento

2023/10/03 14:24:03 INFO mlflow.tracking.fluent: Experiment with name 'house-prices-eda' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/dgamorim/development/mlflow/notebooks/mlruns/629301400594929119', creation_time=1696353843836, experiment_id='629301400594929119', last_update_time=1696353843836, lifecycle_stage='active', name='house-prices-eda', tags={}>

## Linear Regresion

In [74]:
mlflow.start_run()
# ele vai começar a trackear o nosso experiment

<ActiveRun: >

In [75]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [76]:
mlflow.sklearn.log_model(lr, 'lr')
# aqui ele começa a logar o meu modelo para dentro da plataforma do mlflow

/home/dgamorim/development/mlflow/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/dgamorim/development/mlflow/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [77]:
lr_predict = lr.predict(X_test)
lr_predict[0:2]

array([119279.7701544 , 289022.71460536])

In [78]:
X_test.iloc[0]

tamanho      99.0
ano        1963.0
garagem       1.0
Name: 892, dtype: float64

In [79]:
y_test.head(1)

892    154500
Name: preco, dtype: int64

In [80]:
from sklearn.metrics import mean_squared_error, r2_score
import math

In [81]:
mse = mean_squared_error(y_test, lr_predict)
rmse = math.sqrt(mse)
r2 = r2_score(y_test, lr_predict)
metrics = {
    'mse' : mse,
    'rmse' : rmse,
    'r2' : r2
}
metrics

{'mse': 2078666917.9289908,
 'rmse': 45592.39978251848,
 'r2': 0.7021153642898048}

In [82]:
mlflow.log_metrics(metrics)
# logando a métrica do experimento da regressão linear

In [83]:
mlflow.end_run()
# parando a execução do experimento da regressão linear

## Xgboost

In [84]:
from xgboost import XGBRFRegressor, XGBRegressor

In [85]:
xgb_params = {
    'learning_rate' : 0.2,
    'n_estimators' : 50,
    'random_state' : 42
}

with mlflow.start_run():
    xgb = XGBRegressor(**xgb_params)
    xgb.fit(X_train, y_train)
    xgb_predict = xgb.predict(X_test)
    mse = mean_squared_error(y_test, xgb_predict)
    rmse = math.sqrt(mse)
    r2 = r2_score(y_test, xgb_predict)
    metrics = {
    'mse' : mse,
    'rmse' : rmse,
    'r2' : r2
    }
    mlflow.xgboost.log_model(xgb, 'xgboost')
    mlflow.log_metrics(metrics)

/home/dgamorim/development/mlflow/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/dgamorim/development/mlflow/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/dgamorim/development/mlflow/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/dgamorim/development/mlflow/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtyp

/home/dgamorim/development/mlflow/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/dgamorim/development/mlflow/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/dgamorim/development/mlflow/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/dgamorim/development/mlflow/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtyp

## Mlflow API

In [86]:
mlflow.get_experiment_by_name('house-prices-eda')

<Experiment: artifact_location='file:///home/dgamorim/development/mlflow/notebooks/mlruns/629301400594929119', creation_time=1696353843836, experiment_id='629301400594929119', last_update_time=1696353843836, lifecycle_stage='active', name='house-prices-eda', tags={}>


In [88]:
mlflow.

AttributeError: module 'mlflow' has no attribute 'list_run_infos'